In [1]:
datalist = list()
with open("inst6502_comp.txt", "r") as f:
    for line in f:
        data = line.strip().split('\t')
        datalist.extend([i.split(" ",1) for i in data])

datatypes = list(set([i[1] for i in datalist]))
print(datatypes)
#print(len(datatypes))
with open("../debug_itab.z80", "w") as f:
    f.write("inst6502_opcode_width_table:\n")
    for idx, i in enumerate(datatypes):
        size = 0
        if i in ("", "impl", "A"):
            size = 1
        elif i in ("zpg", "rel", "X,ind", "ind,Y", "#", "zpg,X", "zpg,Y"):
            size = 2
        elif i in ("abs", "abs,X", "abs,Y", "ind"):
            size = 3
        pass
        s = f".db {size}    ;{idx}: {i}\n"
        f.write(s)
    f.write("\n\n;Note: 1=1byteOper, 2=2byteOper, -=1byteRelOper. Other chrs are literal\n")
    f.write("\ninst6502_disassembler:\n")
    for idx, i in enumerate(datatypes):
        if i == "":
            s = ""
        elif i == "impl":
            s = ""
        elif i == "A":
            s = "A"
        elif i == "#":
            s = "#1"
        elif i == "zpg":
            s = "1"
        elif i == "rel":
            s = "-"
        elif i == "zpg,X":
            s = "1,X"
        elif i == "zpg,Y":
            s = "1,Y"
        elif i == "X,ind":
            s = "(1,X)"
        elif i == "ind,Y":
            s = "(1),Y"
        elif i == "ind":
            s = "(2)"
        elif i == "abs":
            s = "2"
        elif i == "abs,X":
            s = "2,X"
        elif i == "abs,Y":
            s = "2,Y"
        size = len(s)+1
        f.write(f'.db {size},"{s}"  ;\n')
        


    f.write("\ninst6502_disassembly_table:\n")
    for idx, i in enumerate(datalist):
        s = f'.db "{i[0].strip()[:3]}", {datatypes.index(i[1])}     ;0x{idx:02X}\n'
        f.write(s)





['', 'zpg,X', 'ind', 'abs', 'abs,X', 'X,ind', 'A', 'zpg', 'rel', '#', 'impl', 'ind,Y', 'abs,Y', 'zpg,Y']
